This script was used to scrape data from twitter using IDs given in files NLP+CSS_2016.csv and NAACL_SRW_2016.csv

Import the necessary packages

In [1]:
import numpy as np
import pandas as pd
import tweepy
import pprint, time
from math import ceil

Import the original csv file from the GitHub account, which is in this folder.

In [22]:
# hs = pd.read_csv('./NAACL_SRW_2016.csv', index_col=0, header=None)
hs = pd.read_csv('./NLP+CSS_2016.csv',sep = '\s')
hs = hs.reset_index(drop=True)

/home/rajat/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


Rename column headings, reset ID to not be the ID number

In [26]:
hs = hs.rename(columns={'TweetID': 'ID', 'Expert': 'Class'})
hs.Class = hs.Class.astype('str')
#print(hs.head())

Determine how big the dataset is (rows, columns)

In [27]:
len(hs)

6909

Query the Twitter API to get the tweets from the IDS. Get various key's from Twitter developer website (dev.twitter.com). Run 167 loops of 100 tweets each: first, subset the dataframe by the 100 tweets of each iteration (line 14). Then, define an object "tweets" which actually queries the Twitter API for the IDs (fed in list form) of that subset of tweets. Then add those tweets to an existing list, tweet_list, initialized to an empty list in line 10. Finally, sleep for 60 seconds between each loop to avoid breaking the API rate limit. 

In [36]:
%%time

consumer_key = 'hQSfuYsO9ybVNVKPCxURndKQ6'
consumer_secret = 'BUbTUcCSOozzU3PzFSbt44A4nWDjWVccAwuyPTgzQo25ImhcsV'
access_token = '1030700072755855361-1bl3CyvIAvr8VQ74j368130fq7odUT'
access_token_secret = '1cw58KNSxH0VCs0bqZCbYdevCIm6sIpzQLkuBvglqirwQ'
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

tweet_list = []


for i in range(0, ceil(len(hs.ID) / 100)):
    ids = hs.ID.iloc[i*100 : (i+1)*100]
    tweets = api.statuses_lookup(list(ids), True)
    tweet_list.extend(list(tweets))
    time.sleep(60)

CPU times: user 4.08 s, sys: 425 ms, total: 4.5 s
Wall time: 1h 11min 26s


Choose one tweet and print an object from that tweet for testing purposes, here the first tweet t and its object "text." 

In [37]:
df = pd.DataFrame(tweet_list)

After you've run the querying code for at least three hours, you can create lists of various objects from the tweets. In this function we add all of the attributes we were interested in for this project. To see all of the possible attributes, check out the Twitter API's website.

In [40]:
def get_tweet_info(tweet_list):
    len_list = len(tweet_list)
    tweet = []
    id_str = []
    favorite_count = []
    created_at = []
    screen_name = []
    location = []
    description = []
    followers_count = []
    hashtags = []
    user_mentions = []
    for t in tweet_list:
        tweet.append(t.text)
        id_str.append(t.id_str)
        favorite_count.append(t.favorite_count)
        created_at.append(t.created_at)
        screen_name.append(t.author.screen_name)
        location.append(t.author.location)
        description.append(t.author.description)
        followers_count.append(t.author.followers_count)
        hashtags.append(t.entities['hashtags'])
        user_mentions.append(t.entities['user_mentions'])
    tweet_info = pd.DataFrame({'Tweets': tweet, "ID": id_str, 
                               "Authors": screen_name, "Locations": location, 
                               "Descriptions": description,"Follower Count":followers_count, 
                               "Time Tweeted": created_at, "User Mentions": user_mentions, 
                               "Hashtags": hashtags, "Favorite Count":favorite_count})
    return tweet_info

In [41]:
df = get_tweet_info(tweet_list)
#df.head

And authors. Here I've also included code for how to write the list to a csv file so that we can use it later.

In [43]:
df.to_csv('hatespeech_NLP+CSS.csv')